#Parte 1: Conceptos de RAG
Definición y Aplicación:

Explica qué es el concepto de RAG (Retrieval-Augmented Generation) y cómo se diferencia de un chatbot generativo simple. Da un ejemplo práctico donde un chatbot RAG sería más útil que uno basado solo en generación.
Preguntas de Selección Múltiple.



##Responde:

Respuesta: RAG (Retrieval-Augmented Generation) es un enfoque en inteligencia artificial que combina la generación de texto con la recuperación de información. A diferencia de un chatbot generativo simple que crea respuestas solo a partir de los patrones aprendidos durante su entrenamiento, un modelo RAG primero recupera información relevante de una base de datos o fuente de conocimiento (por ejemplo, documentos, bases de datos, páginas web) y luego genera una respuesta utilizando tanto esa información como su capacidad de generación.

Diferencias clave:
Chatbot generativo simple: Solo se basa en el modelo entrenado para generar respuestas. Está limitado por lo que aprendió en el momento del entrenamiento.
Chatbot RAG: Combina la generación con la recuperación de información externa, lo que le permite generar respuestas más actualizadas y específicas. El componente de recuperación busca datos relevantes para enriquecer la generación.
Ejemplo práctico:
Imagina un asistente de soporte técnico que ayuda a los empleados de una empresa. Un chatbot generativo simple puede ofrecer respuestas generales basadas en problemas comunes, pero puede quedarse corto si el empleado tiene un problema específico o reciente con un sistema que no estaba disponible durante el entrenamiento del modelo.

Un chatbot RAG, en cambio, podría buscar en la base de conocimientos actualizada de la empresa o en la documentación técnica más reciente y ofrecer una solución precisa que involucra la versión exacta del software o sistema que el empleado está usando. Esto sería mucho más útil en casos donde la información cambia con frecuencia.

#Parte 2: Implementación de un Chatbot RAG
Carga y Preparación del Dataset:

Crea y sube un archivo CSV con preguntas frecuentes de un centro educativo llamado Unicuces. El archivo debe contener dos columnas: Pregunta y Respuesta.
Usa pandas para cargar el archivo y convertirlo en un diccionario de preguntas y respuestas.

Instrucciones:

Subir el archivo preguntas_respuestas_unicuces.csv.

In [33]:
!pip install transformers faiss-gpu datasets

In [3]:

# Instalar dependencias
!pip install faiss-cpu
!pip install flask-cors
!pip install pyngrok

from flask_cors import CORS
import threading
import os
import pandas as pd
from flask import Flask, request, jsonify
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
from pyngrok import ngrok

# Inicializar Flask
app = Flask(__name__)
CORS(app)  # Habilitar CORS para evitar restricciones de origen cruzado

# Definir el diccionario de preguntas y respuestas (FAQ)
faq_dict = {
    "¿Qué es la inteligencia artificial?": "La inteligencia artificial (IA) es una rama de la informática que se centra en la creación de sistemas capaces de realizar tareas que normalmente requieren inteligencia humana.",
    "¿Qué es el aprendizaje supervisado?": "El aprendizaje supervisado es un tipo de algoritmo de IA que se entrena con datos etiquetados, donde el modelo aprende a predecir una salida a partir de las entradas.",
    "¿Cuál es la diferencia entre aprendizaje supervisado y no supervisado?": "El aprendizaje supervisado utiliza datos etiquetados para entrenar un modelo, mientras que el aprendizaje no supervisado trabaja con datos no etiquetados para encontrar patrones o agrupaciones.",
    "¿Qué es un modelo de redes neuronales?": "Una red neuronal es un conjunto de algoritmos diseñados para reconocer patrones y se basa en la estructura y funcionamiento del cerebro humano.",
    "¿Qué es el procesamiento del lenguaje natural (NLP)?": "El procesamiento del lenguaje natural es una rama de la IA que se centra en la interacción entre las computadoras y el lenguaje humano, como la comprensión de texto o el reconocimiento de voz.",
    "¿Cómo funciona el aprendizaje por refuerzo?": "El aprendizaje por refuerzo se basa en un agente que toma acciones en un entorno y recibe recompensas o castigos. El objetivo es maximizar las recompensas a lo largo del tiempo.",
    "¿Qué es RAG en inteligencia artificial?": "RAG (Retrieval-Augmented Generation) es un enfoque que combina la generación de texto con la recuperación de información relevante para proporcionar respuestas más precisas y actualizadas.",
    "¿En qué consiste el machine learning?": "El machine learning es un subcampo de la IA que permite a las máquinas aprender a partir de los datos sin ser explícitamente programadas para realizar una tarea.",
    "¿Qué es un chatbot generativo?": "Un chatbot generativo es un modelo de IA que genera respuestas en función de patrones aprendidos de grandes conjuntos de datos sin depender de información externa.",
    "¿Cuál es la diferencia entre RAG y un chatbot generativo simple?": "RAG combina la generación de respuestas con la recuperación de información externa, mientras que un chatbot generativo simple solo usa la información almacenada en su modelo entrenado."
}

# Preparar el modelo y tokenizer de Hugging Face
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Función para codificar texto usando el modelo preentrenado
def embed_text(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    embeddings = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

# Codificar todas las preguntas
preguntas = list(faq_dict.keys())
embeddings = embed_text(preguntas)

# Crear el índice FAISS para realizar búsquedas de similitud
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Configuración de ngrok (mantén seguro tu token)
ngrok.set_auth_token('2nlzBKSP5BmoGB6eCEo93TDLSwg_4aWxdMjoN1p1k6uj7Gg71')

# Ruta API para recibir las consultas y devolver las respuestas
@app.route('/chatbot', methods=['POST'])
def chatbot_api():
    data = request.get_json()  # Recibe el JSON con la consulta
    consulta = data.get('consulta', '')  # Extraer el texto de la consulta

    # Verifica que el campo 'consulta' esté en el JSON
    if not data or 'consulta' not in data:
        return jsonify({"error": "Solicitud incorrecta, falta el campo 'consulta'"}), 400

    # Codificar la consulta y buscar la pregunta más cercana
    consulta_embedding = embed_text([consulta])
    D, I = index.search(consulta_embedding, k=1)
    pregunta_cercana = preguntas[I[0][0]]
    respuesta = faq_dict[pregunta_cercana]
    # Devolver la respuesta en formato JSON
    return jsonify({'respuesta': respuesta})

# Función para ejecutar el servidor Flask
def run_flask():
    app.run()

# Crear el túnel ngrok y obtener la URL pública
public_url = ngrok.connect(5000)
print(f"El servidor Flask está disponible en: {public_url}")

# Iniciar el servidor Flask en un hilo aparte para no bloquear el Colab
thread = threading.Thread(target=run_flask)
thread.start()



El servidor Flask está disponible en: NgrokTunnel: "https://6895-34-23-32-231.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Identifica una mejora a tu chatbo e implementala

Por ejemplo interfaz con Streamlimit, o telegram


INFO:werkzeug:127.0.0.1 - - [22/Oct/2024 05:29:44] "POST /chatbot HTTP/1.1" 404 -


JSONDecodeError: Expecting value: line 1 column 1 (char 0)